# Project 3: Astronomical Source Detection
## Eric Gonzalez Corona
-------------------------------------------
# Part 2: HST image analysis


## Import numpy and sep

In [ ]:
import numpy as np
import sep

## Follow the same steps,initially,as we did for the tutorial. Once again,note that we 
## are using astropy as opposed to fitsio

In [ ]:
from astropy.io import fits # or should it be import astropy.io
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.] # Set the size: do it once here, and never have to do it again in later in the 
                                        #code

## Open the fits file, which was downloaded from "https://archive.stsci.edu/pub/hlsp/hudf12/"


In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits" # already in same folder
hdu_list = fits.open(fname)
hdu_list.info()

## Image information stored in PRIMARY block, accessed by indexing hdu_list

In [ ]:
image_data = hdu_list["PRIMARY"].data

## close FITS file because stored as a variable

In [ ]:
hdu_list.close()

## showing the data

In [ ]:
# show the image
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=0, vmax=m+s, origin='lower')
plt.colorbar();

#vmin = 0 yields a better looking image, so will keep it as that, that way there is very 
#little grey


plt.savefig('skyview2.png' , bbox_inches="tight",dpi=600)

## Background subtraction time, taking away outliers, black is the outlier
-------
## Mask is a true/false array-boolean, where true = you mask the value, false= we don't mask
## that value.
## I wanted to mask the corners, because the corners are off(all back, nothing there). Make a true false array that is true in the corners but false everywhere else, so that the mask, will mask out corners.This gets rid of some of the noise which makes our data more accuate and easier to count the amount of sources in the corners but false everywhere else, so that the mask will mask out corners. 
## Corners are all at 0, insides are at varing ranges. Taking out everything that is zero takes out corners

In [ ]:
image_data = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data,mask =(image_data==0)) # this gives us the true/false array we want

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)






In [ ]:
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

## show the background

In [ ]:

plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.colorbar();
plt.savefig('background2.png' , bbox_inches="tight",dpi=600)

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('noise2.png' , bbox_inches="tight",dpi=600)

In [ ]:

# subtract the background
data_sub = image_data - bkg

## Object detection
## Notice that in Part 2 out sigma value(threshold) is now set to 5, I tried several values(guess and check ) and noticed that it yielded the best answer.

In [ ]:
objects = sep.extract(data_sub, 5, err=bkg.globalrms)

# how many objects were detected
print(len(objects))


from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=0, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)


## Visually, the image on top looks pretty good, previously If I tried the exact same method as the tutorial is was just a sea of red, it was circling way too much. Using the technique/code that I did yields a nice result that tells us that there are a total(approxmately) 1353 objects. 
## Its important to note that increasing or decreasing sigma is one of the factors in determining how accurate the code detects the amount of objects in this image.
-------------


# Now we display the flux of all the objects it detected which stored an an array.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)
    
    

In [ ]:
print(flux)
len(flux) # tells us how many objects it has the flux of, this is another indication of
            #how many objects there are

## #This histogram, shows us that we have mostly faint stars in the image, 0-1 flux, and very few promininet bright objects
## We ignored outliers by making the range= 0-5
## This histograms shows us how the flux is acting in general for the image at hand

In [ ]:
plt.hist(flux,bins = 100,range = [0,5])
plt.xlabel("Flux")
plt.show()